In [ ]:
!pip install transformers sentence-transformers gensim nltk scikit-learn


In [ ]:
import numpy as np

import nltk
# Download the 'punkt_tab' data package
nltk.download('punkt_tab')

from transformers import AutoModelForCausalLM, AutoTokenizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer


In [ ]:
# List of labels

#labels = ["Hypolimnas", "Misippus", "Danaus", "Chrysippus", "Amauris", "Ochlea", "Acraea", "Egina"]
labels = ["Hypolimnas Misippus", "Danaus Chrysippus", "Amauris Ochlea", "Acraea Egina"]

# Generate Contextual Paragraphs Using an LLM

In [ ]:
# Load the Qwen2.5-1.5B-Instruct model and tokenizer        #most download
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).half().cuda()

# Function to generate text for a label
def generate_paragraph(label):
    prompt = f"Write a descriptive paragraph about {label}."
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=250, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Generate paragraphs for each label
contextual_paragraphs_Qwen1_5B = {label: generate_paragraph(label) for label in labels}

# Print the generated paragraphs
for label, paragraph in contextual_paragraphs_Qwen1_5B.items():
    print(f"{label}: {paragraph}\n")


In [ ]:
'''
from transformers import pipeline

# Load a GPT-Neo model for text generation
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B", device=0)

# Define your labels
labels = ["cat", "dog", "bird"]

# Generate contextual paragraphs for each label
contextual_paragraphs_gpt_neo = {}
for label in labels:
    prompt = f"Write a descriptive paragraph about {label}."
    response = generator(prompt, max_length=100, num_return_sequences=1,truncation=True)
    contextual_paragraphs_gpt_neo[label] = response[0]['generated_text']

# Print results
for label, text in contextual_paragraphs_gpt_neo.items():
    print(f"{label}: {text}\n")
'''

In [ ]:
'''
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the NuExtract-1.5-smol model and tokenizer
model_name = "numind/NuExtract-1.5-smol"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).cuda()

# Function to generate text for a label
def generate_paragraph(label):
    prompt = f"Write a detailed and descriptive paragraph about {label}."
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# List of labels
labels = ["cat", "dog", "bird"]

# Generate paragraphs for each label
contextual_paragraphs_numind = {label: generate_paragraph(label) for label in labels}

# Print the generated paragraphs
for label, paragraph in contextual_paragraphs_numind.items():
    print(f"{label}: {paragraph}\n")
'''

#  Prepare Corpus for Word Embedding

Combine the generated paragraphs into a single corpus

In [ ]:
corpus = "\n".join(contextual_paragraphs_Qwen1_5B.values())
print(corpus)


# Generate Word Embeddings

Word2Vec Embeddings

In [ ]:
# Tokenize the corpus into sentences
sentences = [word_tokenize(paragraph.lower()) for paragraph in contextual_paragraphs_Qwen1_5B.values()]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Retrieve embeddings for each label
label_embeddings = {label: word2vec_model.wv[label] for label in labels}
print(label_embeddings)

BERT Embeddings

In [ ]:
# Load a pre-trained BERT model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each label's paragraph
label_embeddings_b = {label: bert_model.encode(paragraph) for label, paragraph in contextual_paragraphs_Qwen1_5B.items()}
print(label_embeddings_b)


In [ ]:
# Aggregate embeddings by averaging
final_vector = np.mean(list(label_embeddings.values()), axis=0)
print("Final Word2Vec Vector Shape:", final_vector.shape)

In [ ]:
# Aggregate embeddings by averaging
final_vector = np.mean(list(label_embeddings_b.values()), axis=0)
print("Final BERT Vector Shape:", final_vector.shape)

In [ ]:
'''
# Save final vector
np.save("final_vector.npy", final_vector)
'''

In [ ]:
'''
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Extract embeddings
embeddings = list(label_embeddings.values())
label_names = list(label_embeddings.keys())

# Reduce dimensionality
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

# Plot the embeddings
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], marker='o')

for i, label in enumerate(label_names):
    plt.annotate(label, (reduced_embeddings[i, 0], reduced_embeddings[i, 1]))

plt.title("Label Embeddings Visualization")
plt.show()
'''